# Initializing 

In [ ]:
import time 

start = time.time()

outputDirectory = '~/Code/brian2cuda/CUBA_CUDA_sp'

In [ ]:
import matplotlib
matplotlib.use('Agg')

from brian2 import *

import brian2cuda
#set_device('cpp_standalone', directory = None, build_on_run=False)
#prefs.devices.cpp_standalone.openmp_threads = 12
#set_device('cpp_standalone', directory = None, compile=True, run=True, debug=True)

set_device('cuda_standalone', directory = None, compile=True, run=True, debug=True)
#set_device('cuda_standalone', build_on_run=False)
#set_device('cuda_standalone', directory=None, build_on_run=False, with_output=False)          

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import seaborn as sns 

In [1]:
%matplotlib inline
%config Completer.use_jedi = False 

/home/user1/.pyenv/versions/2.7.18/envs/py2_brianBrunel/lib/python2.7/site-packages/ipykernel_launcher.py:2: UserWarning: Config option `use_jedi` not recognized by `IPCompleter`.
  


#### Personal Functions 

In [ ]:
def visualise_connectivity(S):
    Ns = len(S.source)
    Nt = len(S.target)
    figure(figsize=(10, 4))
    subplot(121)
    plot(zeros(Ns), arange(Ns), 'ok', ms=10)
    plot(ones(Nt), arange(Nt), 'ok', ms=10)
    for i, j in zip(S.i, S.j):
        plot([0, 1], [i, j], '-k')
    xticks([0, 1], ['Source', 'Target'])
    ylabel('Neuron index')
    xlim(-0.1, 1.1)
    ylim(-1, max(Ns, Nt))
    subplot(122)
    plot(S.i, S.j, 'ok')
    xlim(-1, Ns)
    ylim(-1, Nt)
    xlabel('Source neuron index')
    ylabel('Target neuron index')

In [ ]:
def plot_spikes(monitor, stateMonitor, rangeNeurons):
    for i in range(rangeNeurons):
        spikes = (monitor.t[monitor.i == i] - defaultclock.dt)/ms
        val = stateMonitor[i].V
        subplot(rangeNeurons, 1, i+1)
        plot(tile(spikes, (2, 1)), vstack((val[array(spikes, dtype=int)], zeros(len(spikes)))), 'C0')
        title("Spikes")
    tight_layout()
    show()

In [ ]:
def plot_potential(monitor, stateMonitor, rangeNeurons):
    for i in range(rangeNeurons):
        spikes = (monitor.t[monitor.i == i] - defaultclock.dt)/ms
        val = stateMonitor[i].V
        subplot(rangeNeurons, 1, i+1)
        plot(stateMonitor.t/ms, val)
        title("Trace")
    tight_layout()
    show()

In [ ]:
def plot_population(prMonitorRun, spikeMonitorRun, labelGroup, nActivity, pRun): 
    # plotting
    title('Population rates')
    xlabel('ms')
    ylabel('Hz')

    plot(prMonitorRun.t / ms, prMonitorRun.smooth_rate(width=25 * ms) / Hz, label=labelGroup)

    legend()
    figure()

    title('Population activities ({} neurons/pop)'.format(nActivity))
    xlabel('ms')
    yticks([])

    plot(spikeMonitorRun.t / ms, spikeMonitorRun.i + (pRun + 1) * nActivity, '.', markersize=2, label=labelGroup) 


    legend()
    show()

In [ ]:
def plot_population_Multi(prMonitorRun_smooth, prMonitorRun_t, spikeMonitorRun, labelGroup, nActivity, pRun): 
    # plotting
    title('Population rates')
    xlabel('ms')
    ylabel('Hz')

    plot(prMonitorRun_t / ms, prMonitorRun_smooth, label=labelGroup)

    legend()
    figure()

    title('Population activities ({} neurons/pop)'.format(nActivity))
    xlabel('ms')
    yticks([])

    plot(spikeMonitorRun.t / ms, spikeMonitorRun.i + (pRun + 1) * nActivity, '.', markersize=2, label=labelGroup) 

    legend()
    show()

In [ ]:
def plot_population_Multi_2(prMonitorRun_smooth, prMonitorRun_t, labelGroup): 
    # plotting
    title('Population rates')
    xlabel('ms')
    ylabel('Hz')

    plot(prMonitorRun_t / ms, prMonitorRun_smooth, label=labelGroup)

    legend()
    show()

In [ ]:
def active_percent(monitor, binSize, durationSimulation, numNeurons, ymax):
    # Define bin size 
    bin_size = binSize*ms
    # Define duration of simulation 
    duration = durationSimulation*second
    # Define number of neurons 
    num_Neurons = numNeurons

    # Initialize array to record spike occurancces 
    spkTotal = np.zeros([int(duration/bin_size)])
    
    for current in range(num_Neurons): 
        spk_count, bin_edges = np.histogram(a = np.r_[monitor.t[monitor.i == current]/second],
                                            bins = int(duration/bin_size), range = (0,durationSimulation))
        # Convert spike count to binary 
        binary = np.where(spk_count > 0, 1, 0)
        # Add to recording array
        spkTotal = spkTotal + binary

    plt.plot(bin_edges[:-1], (spkTotal/num_Neurons)*100)
    plt.axis([0, durationSimulation, 0, ymax])
    plt.title('Percent active granule cells as function of time')
    plt.ylabel('Active Cell (%)')
    plt.xlabel('Time (s)')
    plt.show()

In [ ]:
def averaged_ActivePercent(spkMulti, spkSingle, binSize, durationSimulation, numNeurons, ymax):
    # Define bin size 
    bin_size = binSize*ms
    # Define duration of simulation 
    duration = durationSimulation*second
    # Define number of neurons 
    num_Neurons = numNeurons
    
    spk_count, bin_edges = np.histogram(a = np.r_[spkSingle.t[spkSingle.i == 1]/second],
                                        bins = int(duration/bin_size), range = (0,durationSimulation))

    averaged_spkMulti = np.mean(spkMulti, axis = 0)
    
    plt.plot(bin_edges[:-1], averaged_spkMulti)
    plt.axis([0, durationSimulation, 0, ymax])
    plt.title('Percent active granule cells as function of time')
    plt.ylabel('Active Cell (%)')
    plt.xlabel('Time (s)')
    plt.show()

#### General paramaters 

In [ ]:
# spatial 
#gC = 320
gC = 10
numGC = gC**2 
gG = 32 
#numGolgi = gG**2  
numGolgi = 10 
gGlo = 31
numGlomeruli = gGlo**2

# Spatial paramaters 
gcSpacing = ((gG*(gG-1))/(gC-1))*umetre 
golgiSpacing = 32*umetre 

gcSpacing = ((gG*(gG-1))/(gC-1))*umetre 

# Neuron Models

#### Granule Cells 

In [ ]:
# Parameters 

# neuron parameters
theta_GR = -35*mV # threshold (from Yamazaki) 
Cm_GR = 3.1*pF 

# Conductances 
gL_GR = 0.43*nS 
g_AMPA_GR = 0.18*nS 
g_NMDA_GR = 0.025*nS 
g_Inh_GR = 0.028*nS 
g_Ahp_GR = 1*nS

# Various leaks (from Yamazaki) 
El_GR = -58*mV
Eexc = 0*mV 
E_inh_GR = -82*mV 
E_ahp_GR = -82*mV # after hyperpolarization 

# Tau
tau_ahp_GR = 5*ms 
tau_AMPA_GR = 1.2*ms 
tau_NMDA_GR = 52.0*ms 
tau_GABA_1_GR = 7*ms
tau_GABA_2_GR = 59*ms

eqs_GR = Equations('''
                        dV / dt = (-gL_GR * (V - El_GR) - I_syn) / Cm_GR : volt (unless refractory)
                        
                        I_syn = I_AMPA + I_NMDA + I_GABA_tot : amp 
                        I_GABA_tot = 0.43*I_GABA_1 + 0.57*I_GABA_2 : amp 
                        
                        I_AMPA = g_AMPA_GR * (V - Eexc) * s_AMPA : amp
                        ds_AMPA / dt = -s_AMPA / tau_AMPA_GR : 1 
                        
                        I_NMDA = g_NMDA_GR * (V - Eexc) * s_NMDA : amp
                        ds_NMDA / dt = -s_NMDA / tau_NMDA_GR : 1 
                        
                        I_GABA_1 = g_Inh_GR * (V - E_inh_GR) * s_GABA_1 : amp
                        ds_GABA_1 / dt = -s_GABA_1 / tau_GABA_1_GR : 1 
                        
                        I_GABA_2 = g_Inh_GR * (V - E_inh_GR) * s_GABA_2 : amp
                        ds_GABA_2 / dt = -s_GABA_2 / tau_GABA_2_GR : 1 
                        
                        x : metre 
                        y : metre 
                ''')

# Defining granule cell neuron group 
granuleCells = NeuronGroup(N=numGC, model=eqs_GR, reset='V = El_GR',
                    threshold='V > theta_GR',
                    refractory=2*ms, method='euler')
granuleCells.V = El_GR 

In [ ]:
granuleCells

In [ ]:
# Defining MF input 

# Defining input parameters 
num_inputs = 100
f = 0.5*Hz 
eq_rate = '(22.5 + 7.5*cos((2*pi*t*f) + pi))*Hz'

# Equations 
eqs_glut_MFGR = '''
               w_MFGR : 1 
               '''
eqs_pre_glut_MFGR = '''
               s_AMPA += w_MFGR
               s_NMDA += w_MFGR
               '''

P_MFGR = PoissonGroup(num_inputs, rates = eq_rate)
S_MFGR = Synapses(P_MFGR, granuleCells, model = eqs_glut_MFGR, on_pre = eqs_pre_glut_MFGR, method = 'euler') 
S_MFGR.connect(condition = 'i == j')
S_MFGR.w_MFGR = 4



In [ ]:
# Visualize synaptic connections 
# visualise_connectivity(S_MFGR)

#### Golgi

In [ ]:
# Parameters 

# neuron parameters
theta_GO = -52*mV # threshold (from Yamazaki) 
Cm_GO = 28.0*pF 

# Conductances 
gL_GO = 2.3*nS 
g_AMPA_GO = 45.5*nS 
g_NMDA_GO = 30.0*nS 
g_Ahp_GO = 20*nS

El_GO = -55*mV
Eexc = 0*mV 
E_ahp_GO = -72.7*mV # after hyperpolarization 

# Tau
tau_ahp_GO = 5*ms 
tau_AMPA_GO = 1.5*ms 
tau_NMDA_1_GO = 31.0*ms 
tau_NMDA_2_GO = 170.0*ms 

eqs_GO = Equations('''
                        dV / dt = (-gL_GO * (V - El_GO) - I_syn) / Cm_GO : volt (unless refractory)
                        
                        I_syn = I_AMPA + I_NMDA_tot : amp 
                        I_NMDA_tot = 0.33*I_NMDA_1 + 0.67*I_NMDA_2 : amp
                        
                        I_AMPA = g_AMPA_GR * (V - Eexc) * s_AMPA : amp
                        ds_AMPA / dt = -s_AMPA / tau_AMPA_GO : 1 
                        
                        I_NMDA_1 = g_NMDA_GO * (V - Eexc) * s_NMDA_1 : amp
                        ds_NMDA_1 / dt = -s_NMDA_1 / tau_NMDA_1_GO : 1 
                        
                        I_NMDA_2 = g_NMDA_GO * (V - Eexc) * s_NMDA_2 : amp
                        ds_NMDA_2 / dt = -s_NMDA_2 / tau_NMDA_2_GO : 1 
                        
                        x : metre 
                        y : metre 
                ''')

# Defining granule cell neuron group 
golgi = NeuronGroup(N=numGolgi, model=eqs_GO, reset='V = El_GO',
                    threshold='V > theta_GO',
                    refractory=5*ms, method='euler')
golgi.V = El_GO 

In [ ]:
golgi

#  Establishing Connectivity 

#### Paramaters 

In [ ]:
# Connection probability 
pr_GRGO = 0.0245
#pr_GOGR = 0.025
pr_GOGR = 0.08

#### Granule --> Golgi

In [ ]:
# Equations 
eqs_glut_GRGO = '''
               w_GRGO : 1 
               '''
eqs_pre_glut_GRGO = '''
               s_AMPA += 1
               s_NMDA_1 += 1
               s_NMDA_2 += 1 
               '''

In [ ]:
# Forming synapses 
S_GRGO = Synapses(granuleCells, golgi, model = eqs_glut_GRGO, on_pre = eqs_pre_glut_GRGO, method = 'euler') 
S_GRGO.connect(p = pr_GRGO)

In [ ]:
#visualise_connectivity(S_GRGO)

#### Golgi --> Granule Cells 

In [ ]:
# Equations 
eqs_GABA_GOGR = '''
               w_GOGR : 1 
               '''
eqs_pre_GABA_GOGR = '''
               s_GABA_1 += 1
               s_GABA_2 += 1 
               '''

In [ ]:
# Forming synapses 
S_GOGR = Synapses(golgi, granuleCells, model = eqs_GABA_GOGR, on_pre = eqs_pre_GABA_GOGR, method = 'euler') 
#S_GOGR.connect(condition = 'abs(i-j)<2')
S_GOGR.connect(p = pr_GOGR)

In [ ]:
#visualise_connectivity(S_GOGR)

# Running simulation 

In [ ]:
def spk_active(monitor, binSize, durationSimulation, numNeurons): 
    # Define bin size 
    bin_size = binSize*ms
    # Define duration of simulation 
    duration = durationSimulation*second
    # Define number of neurons 
    num_Neurons = numNeurons

    # Initialize array to record spike occurancces 
    spkTotal = np.zeros([int(duration/bin_size)])
    
    for current in range(num_Neurons): 
        spk_count, bin_edges = np.histogram(a = np.r_[monitor.t[monitor.i == current]/second],
                                            bins = int(duration/bin_size), range = (0,2))
        # Convert spike count to binary 
        binary = np.where(spk_count > 0, 1, 0)
        # Add to recording array
        spkTotal = spkTotal + binary
    
    return((spkTotal/num_Neurons)*100)

In [ ]:
time.time() - start 

In [ ]:
time.time() - start 

In [ ]:
device.reinit()
device.activate()
#set_device('cuda_standalone', directory = None, compile=True, run=True, debug=True)

In [ ]:
run(2*second)
run(2*second)

device.build(directory=None, compile=True, run=True, debug=False, clean=True, direct_call=False)

In [ ]:
time.time() - start 

In [ ]:
# run the simulation
spikeMonitor_GR = SpikeMonitor(granuleCells)

stateMonitor_GR = StateMonitor(granuleCells, 'V', record=True)
prMonitor_GR = PopulationRateMonitor(granuleCells)

#BrianLogger.log_level_info()

#net = Network(collect())
# Snapshots of state 
#net.store('initial') 

spikeMonitor_Multi_GR = []
spkActive_Multi_GR = []
populationRates_GR = []
prMulti_GR_smooth = []
prMulti_GR_t = []

repeatSimulation = 1


itter1 = 0
itter2 = 0 
itter3 = 0 
itter4 = 0 


for trial in range(repeatSimulation): 
    itter1 += 1
    #net.restore('initial')
    
    run(2*second)
    itter2 += 1
    
    device.build(directory=outputDirectory, compile=True, run=True, debug=False)
    #net.run(2*second, report ='stdout', profile = True)
    
    device.reinit()
    device.activate(build_on_run=False)
    
    itter3 += 1
    
    
    itter4 += 1

In [ ]:
time.time() - start 

In [ ]:
?device.activate()

In [ ]:
?device.build()

In [ ]:
?set_device()

In [ ]:
itter1 

In [ ]:
itter2

In [ ]:
itter3

In [ ]:
itter4

In [ ]:

    spikeMonitor_Multi_GR.append(spikeMonitor_GR.spike_trains())
    spkActive_Multi_GR.append(spk_active(spikeMonitor_GR, binSize = 10, durationSimulation = 2, numNeurons = numGC))
    
    populationRates_GR.append(prMonitor_GR)
    prMulti_GR_smooth.append(prMonitor_GR.smooth_rate(width = 25* ms) / Hz)
    prMulti_GR_t.append(prMonitor_GR.t) 

In [ ]:
# For benchmarking time usage per step ... do not use with multiple runs 
profiling_summary(show = 10)

# Visualization 

In [ ]:
plot_potential(spikeMonitor_GR, stateMonitor_GR, 2)

In [ ]:
plot_spikes(spikeMonitor_GR, stateMonitor_GR, 2)

In [ ]:
plot_population(prMonitor_GR, spikeMonitor_GR, 'GranuleCells', 10, 10)

In [ ]:
plot_population_Multi(prMulti_GR_smooth[0], prMulti_GR_t[0], spikeMonitor_GR, 'GranuleCells', 10, 10)

In [ ]:
plot_population_Multi(prMulti_GR_smooth[1], prMulti_GR_t[1], spikeMonitor_GR, 'GranuleCells', 10, 10)

In [ ]:
tempAppend = []
neuronNumber = 0

for i in range(len(spikeMonitor_Multi_GR)):
    tempAppend.extend(pd.array(spikeMonitor_Multi_GR[i][neuronNumber])*1000)

sns.kdeplot(tempAppend, bw_adjust = 0.5)

In [ ]:
tempAppend = []
neuronNumber = 1

for i in range(len(spikeMonitor_Multi_GR)):
    tempAppend.extend(pd.array(spikeMonitor_Multi_GR[i][neuronNumber])*1000)

sns.kdeplot(tempAppend, bw_adjust = 0.2)

In [ ]:
tempAppend = []
neuronNumber = 2

for i in range(len(spikeMonitor_Multi_GR)):
    tempAppend.extend(pd.array(spikeMonitor_Multi_GR[i][neuronNumber])*1000)

sns.kdeplot(tempAppend, bw_adjust = 0.2)

In [ ]:
tempAppend = []
neuronNumber = 19

for i in range(len(spikeMonitor_Multi_GR)):
    tempAppend.extend(pd.array(spikeMonitor_Multi_GR[i][neuronNumber])*1000)

sns.kdeplot(tempAppend, bw_adjust = 0.1)

#### Population Readouts

In [ ]:
# Plotting average population firing rate across all runs 
average_prMulti = np.mean(prMulti_GR_smooth, axis = 0)

plot_population_Multi_2(average_prMulti, prMulti_GR_t[1], 'GranuleCells')

In [ ]:
active_percent(spikeMonitor_GR, binSize = 10, durationSimulation = 2, numNeurons = numGC, ymax = 20)

In [ ]:
averaged_ActivePercent(spkActive_Multi_GR, spikeMonitor_GR, binSize = 10, durationSimulation = 2, numNeurons = numGC, ymax = 15)